In [ ]:
import pandas as pd, numpy as np
import ast

In [339]:
dataframe = pd.read_csv("../scraping/out.csv")

In [346]:
dataframe.columns

Index(['title', 'singers', 'directors', 'lyricist', 'genre', 'album',
       'download_link', 'poster', 'year'],
      dtype='object')

In [345]:
dataframe.shape

(1375, 9)

In [349]:
dataframe.tail()

,title,singers,directors,lyricist,genre,album,download_link,poster,year
1370,You And Me - Teri Galiyan,"['Kabuki Khanna', ' Ankit Tiwari']",['Ankit Tiwari'],['Manoj Muntashir'],"['Filmi', ' Pop']",You And Me - Teri Galliyan,https://www.youtube.com/watch?v=yK1KJEAGtyU,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
1371,Zara Sun Balliye,['Krish - II'],['Kartikeya Tiwari'],['Kartikeya Tiwari'],"['Filmi', ' Pop']",Zara Sun Balliye,https://www.youtube.com/watch?v=Kq39voX2K9c,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
1372,Haathon Mein Haath,"['Roop Kumar Rathod', ' Sonali Rathod']",['Roop Kumar Rathod'],['Shakeel Azmi'],['Ghazal'],Zikr Tera,https://www.youtube.com/watch?v=FpQex_c_rFY,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
1373,Zindagi,['Aditya Narayan'],['Harshit Chauhan'],['Prashant Ingole'],['Pop'],Zindagi,https://www.youtube.com/watch?v=3erGbvKJXqo,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
1374,Zindagi Aa Raha Hoon Main,['Atif Aslam'],['Amaal Malik'],['Manoj Muntashir'],['Pop'],Zindagi Aa Raha Hoon Main,https://www.youtube.com/watch?v=82eM7QRtoRo,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015


In [343]:
#checking for null values
null_lyricist = dataframe.isna()['lyricist'].sum()
null_directors = dataframe.isna()['directors'].sum()
print(null_directors, null_lyricist)

7 29


In [344]:
#removing brackets 
dataframe = dataframe[~dataframe['title'].str.contains(r'\(|\)')]

In [348]:
dataframe.dropna(subset=['title','singers','download_link', 'year', 'poster'], inplace= True)
dataframe.reset_index(drop = True, inplace=True)

In [ ]:
def convert_to_words(words):
    try:
        if not words:
            return []
        list = ast.literal_eval(words)
        return " " + " ".join([one_word(s) for s in list])
    except:
        return " "
def one_word(s):
    return "".join(s.lower().split())

In [350]:
new_df = pd.DataFrame()
new_df['tags'] = dataframe['title']
new_df['tags'] += dataframe['singers'].apply(convert_to_words)
new_df['tags'] += dataframe['directors'].apply(convert_to_words)
new_df['tags'] += dataframe['lyricist'].apply(convert_to_words)
new_df['tags'] += dataframe['genre'].apply(convert_to_words)
new_df['tags'] += " " + dataframe['album']

In [ ]:
new_df.head().iloc[0].tags

In [351]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [352]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
type(new_df['tags'])

In [353]:
vector.shape

(1375, 3624)

In [354]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [312]:
similarity.shape

(1375, 1375)

In [355]:
sorted_indices = np.array([np.argsort(-row)[1: 51] for row in similarity])
np.save('similarity.npy', sorted_indices)

In [ ]:
#loading data from numpy
arr = np.load('similarity.npy')
list(arr[0])

In [314]:
def recommend(song):
    index = dataframe[dataframe['title'] == song].index[0]
    similar = enumerate(similarity[index])
    r_songs = sorted(similar, key = lambda x : -x[1])[1:51]
    list_of_songs = []
    for sng in r_songs:
        list_of_songs.append(dataframe.iloc[sng[0]].to_dict())
    return list_of_songs

In [ ]:
r = recommend(dataframe.iloc[0].title)

In [ ]:
[i['index'] for i in r][:50]

In [356]:
#for saving the database
save_df = pd.DataFrame()
save_df = dataframe[['title', 'download_link', 'poster', 'year']]

In [ ]:
save_df.head()

In [357]:
save_df.to_pickle('songs.pkl')

In [ ]:
#loading dataframe
df = pd.read_pickle('songs.pkl')
df

### User Search recommendation

In [ ]:
user_search = 'aaj na jaana'
song_titles = dataframe['title'].tolist()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
vectorizer = TfidfVectorizer()
all_transform = vectorizer.fit_transform(song_titles)

In [ ]:
query_transform = vectorizer.transform([user_search])

In [ ]:
similar_songs = cosine_similarity(query_transform, all_transform)

In [ ]:
songs = np.argsort(-similar_songs[0]).tolist()[:50]

### homepage songs

In [358]:
songs = dataframe.sort_values(by = "year").iloc[-50:]

In [359]:
songs

,title,singers,directors,lyricist,genre,album,download_link,poster,year
895,Ho Chhoti Si Koi Tension,['Rajnigandha Shekhawat'],['Ashok Mukherjee'],['Ashesh Sharma'],['Filmi'],Dhuntaaraa,https://www.youtube.com/watch?v=S0hSIlizSCM,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
896,Dil Dhadakne Do,"['Priyanka Chopra', ' Farhan Akhtar']",['Shankar - Ehsaan - Loy'],['Javed Akhtar'],['Pop'],Dil Dhadakne Do,https://www.youtube.com/watch?v=56ZzM4mz4yY,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
897,Pehli Baar,"['Sukriti Kakkar', ' Siddharth Mahadevan']",['Shankar - Ehsaan - Loy'],['Javed Akhtar'],"['Pop', ' Rock']",Dil Dhadakne Do,https://www.youtube.com/watch?v=VkJlv0m6els,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
898,Gallan Goodiyaan,"['Manish Tipu', ' Farhan Akhtar', ' Shankar Ma...",['Shankar - Ehsaan - Loy'],['Javed Akhtar'],"['Filmi', ' Bhangra']",Dil Dhadakne Do,https://www.youtube.com/watch?v=jCEdTq3j-0U,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
899,Girls Like To Swing,['Sunidhi Chauhan'],['Shankar - Ehsaan - Loy'],['Javed Akhtar'],"['Pop', ' Jazz']",Dil Dhadakne Do,https://www.youtube.com/watch?v=JTWGBaTV_Ig,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
900,Phir Bhi Yeh Zindagi,"['Farhan Akhtar', ' Vishal Dadlani', ' Divya K...",['Shankar - Ehsaan - Loy'],['Javed Akhtar'],"['Filmi', ' Pop', ' Sugam']",Dil Dhadakne Do,https://www.youtube.com/watch?v=7Ngw_e2uEZ0,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
921,Toote Taare,['Swati Sharrma'],['Raeth'],['A.M. Turaz'],['Pop'],Direct Ishq,https://www.youtube.com/watch?v=8cy9LOMKQdk,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
922,Mera Kissa,['Swati Sharrma'],['Vivek Kar'],['A.M. Turaz'],"['Rock', ' Filmi']",Direct Ishq,https://www.youtube.com/watch?v=wtktz9uzPjQ,https://is1-ssl.mzstatic.com/image/thumb/Music...,2015
923,Mere Ghaghare Ke Liye Ghamasan Machi Hai,['Mamta Sharma'],['Sanjeev - Darshan'],['Sameer'],['Filmi'],Dirty Politics,https://www.youtube.com/watch?v=igwwXv5cuig,https://is4-ssl.mzstatic.com/image/thumb/Music...,2015
924,Chal Dum,['Neha Bhasin'],['Aadesh Shrivastava'],['Sameer'],['Filmi'],Dirty Politics,https://www.youtube.com/watch?v=vm9VUt_imGA,https://is4-ssl.mzstatic.com/image/thumb/Music...,2015
